本代码由kaggle教程改编，来自https://www.kaggle.com/competitions/word2vec-nlp-tutorial/overview

在Word2Vec中 相似词之间的意思，会体现在词向量的距离上

In [1]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )
test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, " \
 "and %d unlabeled reviews\n" % (train["review"].size,  
 test["review"].size, unlabeled_train["review"].size ))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews



In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [3]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download()   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [4]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)
#可以减少训练集方便测试
print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\jackhong\AppData\Local\Temp\ipykernel_4356\1919973641.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()
C:\Users\jackhong\AppData\Local\Temp\ipykernel_4356\1919973641.py:11: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


Parsing sentences from unlabeled set


In [5]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            vector_size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name =("300features_40minwords_10context")
model.save(model_name)

2024-09-16 17:17:41,849 : INFO : collecting all words and their counts
2024-09-16 17:17:41,850 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-09-16 17:17:41,914 : INFO : PROGRESS: at sentence #10000, processed 225664 words, keeping 17775 word types
2024-09-16 17:17:41,980 : INFO : PROGRESS: at sentence #20000, processed 451738 words, keeping 24945 word types
2024-09-16 17:17:42,041 : INFO : PROGRESS: at sentence #30000, processed 670859 words, keeping 30027 word types


Training model...


2024-09-16 17:17:42,117 : INFO : PROGRESS: at sentence #40000, processed 896841 words, keeping 34335 word types
2024-09-16 17:17:42,190 : INFO : PROGRESS: at sentence #50000, processed 1116082 words, keeping 37751 word types
2024-09-16 17:17:42,259 : INFO : PROGRESS: at sentence #60000, processed 1337544 words, keeping 40711 word types
2024-09-16 17:17:42,331 : INFO : PROGRESS: at sentence #70000, processed 1560307 words, keeping 43311 word types
2024-09-16 17:17:42,406 : INFO : PROGRESS: at sentence #80000, processed 1779516 words, keeping 45707 word types
2024-09-16 17:17:42,487 : INFO : PROGRESS: at sentence #90000, processed 2003714 words, keeping 48121 word types
2024-09-16 17:17:42,560 : INFO : PROGRESS: at sentence #100000, processed 2225465 words, keeping 50190 word types
2024-09-16 17:17:42,637 : INFO : PROGRESS: at sentence #110000, processed 2444323 words, keeping 52058 word types
2024-09-16 17:17:42,711 : INFO : PROGRESS: at sentence #120000, processed 2666488 words, keepin

In [6]:
# model.wv.doesnt_match("man woman child kitchen".split()) #'Word2Vec' object has no attribute 'doesnt_match' --> add wv
# model.wv.doesnt_match("france england germany berlin".split())
# model.wv.doesnt_match("paris berlin london austria".split())
# model.wv.most_similar("man")
# model.wv.most_similar("awful")
model.wv.most_similar("queen")

[('princess', 0.6691381931304932),
 ('bride', 0.6222976446151733),
 ('belle', 0.6075000762939453),
 ('maid', 0.5887537002563477),
 ('angela', 0.5815092921257019),
 ('victoria', 0.5757418274879456),
 ('prince', 0.5728840827941895),
 ('stepmother', 0.5666468143463135),
 ('maria', 0.5620967149734497),
 ('mistress', 0.5586085319519043)]

In [7]:
# from gensim.models import Word2Vec
# model = Word2Vec.load("300features_40minwords_10context")
# type(model.syn0) #在 Gensim 3.8.0 版本之后，Word2Vec 类的 syn0 属性已经被移除。
print(type(model.wv.vectors))
print(model.wv.vectors.shape) #行数对应词表中的单词数，列数对应设置的特征向量的大小
#单个单词向量可以通过如下方式访问
print(model.wv["flower"]) # 教程中的model["flower"]会报错，因为gensim新版本不支持

<class 'numpy.ndarray'>
(16490, 300)
[ 0.01343697  0.10499929 -0.04231662  0.08225445  0.03389212  0.01985863
  0.00542829  0.0737689   0.02672863 -0.07274513 -0.038321    0.05418455
 -0.01052933 -0.0411646  -0.04267859 -0.03867198  0.01146441 -0.12895207
 -0.04743868  0.0196787  -0.01693005  0.00748485 -0.01420624  0.0553043
  0.08279351 -0.01862838 -0.0987298   0.0220592  -0.03773619  0.01561076
  0.0283215  -0.03485876 -0.01787732 -0.03470311  0.01913091  0.02364818
  0.06801102 -0.04239033  0.01862859 -0.03524388  0.02706522  0.04168015
  0.08228005 -0.00158841 -0.07270744  0.02245994 -0.06644198  0.07976009
 -0.01845952  0.02333832  0.07308139  0.0852331   0.00209506 -0.02183389
  0.04718404  0.0677022   0.11174063  0.04824248 -0.04721171 -0.09814323
 -0.15274154  0.08277468  0.06711916  0.02600575  0.04297187 -0.0424794
 -0.03792166 -0.05858068 -0.04492789  0.0120226   0.09069689  0.0288656
  0.05190144 -0.05931181  0.01915037 -0.03528772  0.0193501   0.0540079
 -0.08216172  0.01

We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.
Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

In [8]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index_to_key)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model.wv[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors 
       # only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [9]:
# numpy二维矩阵对第1行赋值示例
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])
arr[1]=[10, 11, 12]
print(arr)

[[ 1  2  3]
 [10 11 12]
 [ 7  8  9]]


In [10]:
# 打印词汇表
print(model.wv.key_to_index)#返回字典 key为单词，value为索引
print(model.wv.index_to_key)#返回列表 元素值为单词

{'the': 0, 'and': 1, 'a': 2, 'of': 3, 'to': 4, 'is': 5, 'it': 6, 'in': 7, 'i': 8, 'this': 9, 'that': 10, 's': 11, 'was': 12, 'as': 13, 'with': 14, 'for': 15, 'movie': 16, 'but': 17, 'film': 18, 'you': 19, 't': 20, 'on': 21, 'not': 22, 'he': 23, 'are': 24, 'his': 25, 'have': 26, 'be': 27, 'one': 28, 'all': 29, 'at': 30, 'they': 31, 'by': 32, 'who': 33, 'an': 34, 'from': 35, 'so': 36, 'like': 37, 'there': 38, 'her': 39, 'or': 40, 'just': 41, 'about': 42, 'out': 43, 'has': 44, 'if': 45, 'what': 46, 'some': 47, 'good': 48, 'can': 49, 'more': 50, 'when': 51, 'very': 52, 'she': 53, 'up': 54, 'no': 55, 'time': 56, 'even': 57, 'would': 58, 'my': 59, 'which': 60, 'their': 61, 'story': 62, 'only': 63, 'really': 64, 'see': 65, 'had': 66, 'were': 67, 'well': 68, 'we': 69, 'me': 70, 'than': 71, 'much': 72, 'bad': 73, 'get': 74, 'been': 75, 'people': 76, 'also': 77, 'into': 78, 'do': 79, 'great': 80, 'other': 81, 'will': 82, 'first': 83, 'because': 84, 'him': 85, 'how': 86, 'most': 87, 'don': 88, 't

In [11]:
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

C:\Users\jackhong\AppData\Local\Temp\ipykernel_4356\1919973641.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review).get_text()


Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 1900

In [12]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [13]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.vectors#model.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = int(num_clusters) )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  152.3914725780487 seconds.


In [14]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index_to_key, idx ))
# print(word_centroid_map)
# print(word_centroid_map.values())
# print(len(word_centroid_map.values()))
# print(len(word_centroid_map))
# print(word_centroid_map.keys()[0]) #报错
word_centroid_keys = list(word_centroid_map.keys())
word_centroid_values = list(word_centroid_map.values())
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_values)):
        if( word_centroid_values[i] == cluster ):
            words.append(word_centroid_keys[i])
    print(words)


Cluster 0
['nighttime']

Cluster 1
['pet', 'homer', 'keeper', 'grandpa', 'rusty', 'timmy', 'fido', 'caretaker', 'puppies', 'handyman', 'gage']

Cluster 2
['scheme', 'crusade', 'ownership', 'newfound']

Cluster 3
['corbett', 'duryea']

Cluster 4
['pandering', 'disingenuous']

Cluster 5
['degradation', 'powerless', 'carnal', 'masculinity', 'treason']

Cluster 6
['sea', 'moon', 'river', 'mountain', 'ocean', 'tower', 'gate', 'pole', 'volcano', 'pond', 'abyss']

Cluster 7
['visual', 'images', 'visuals', 'visually', 'storytelling', 'imagery']

Cluster 8
['faced', 'confronted']

Cluster 9
['anti', 'alternative', 'educational', 'objective', 'injustice', 'allegory']


In [15]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [16]:
# Pre-allocate an array for the training set bags of centroids (for speed) #'float' object cannot be interpreted as an integer-->需要把nums_clusters转成int
train_centroids = np.zeros( (train["review"].size, int(num_clusters)), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, int(num_clusters)), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [17]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
